In [1]:
# Sample version of excel compare for github

# import pandas and numpy
import pandas as pd
import numpy as np

# define the diff function that will highlight any changes
def show_diff(x):
    return x[0] if x[0] == x[1] else'{} ------->{}'.format(*x)


# read in the files

old = pd.read_excel('StudentTestA.xlsx', 'StudentPass', na_values='NA', encoding='utf-8')

new = pd.read_excel('StudentTestB.xlsx', 'StudentPass', na_values='NA', encoding='utf-8')



# Rename the columns, just for practice of renaming, to ensure they are all the same
old = old.rename(columns={'StudentName':'Name', 'StudentID': 'ID'})


new = new.rename(columns={'StudentName':'Name', 'Student Identifier': 'ID'})


# take a subset of the rows we care about
new = new[['Name', 'ID', 'Status']]
old = old[['Name', 'ID', 'Status']]


# add a column to track which version the records are in when we add the dataframes together
old['Version'] = "old"
new['Version'] = 'new'

# consolidate all the data together
full_file = pd.concat([old, new], ignore_index=True)



# see the changes in the main columns we care about
changes = full_file.drop_duplicates(subset=['Name', 'ID', 'Status'], keep= 'last')


# get all the duplicate rows
duplicates = changes.set_index(['ID']).index.get_duplicates()


# get all the duplicates ID
dupes = changes[changes['ID'].isin(duplicates)]

# Create 2 dataframes to split them into the old and new file
changes_new = dupes[(dupes["Version"]=="new")]
changes_old = dupes[(dupes["Version"]=="old")]

#  no longer need the version column in the changes_old/new DF since they're separated out
changes_new = changes_new.drop(["Version"], axis=1) # axis 1 means it will apply to all rows
changes_old = changes_old.drop(["Version"], axis=1)


# index on the ID
changes_old.set_index(["ID"], inplace=True) # inplace=True modifies the existing df rather than create a new object

changes_new.set_index(["ID"], inplace=True)



# create a file to show the old and new changes
changes_old.to_excel('changes_old_ascii.xlsx')
changes_new.to_excel('changes_new_ascii.xlsx')

# run the diff function as the data is now structured in the right way
diff_data = pd.Panel(dict(df1=changes_old, df2=changes_new))

diff_output = diff_data.apply(show_diff, axis=0)
diff_output

,Name,Status
ID,,
9403,VVV,Pending ------->Pass
1065,WWW,Pending ------->Fail


In [2]:
diff_output.to_excel('StudentChanges.xlsx')